In [1]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import accuracy_score
import numpy as np
import math
import random
import warnings
from matplotlib import pyplot as plt
%matplotlib inline

C:\Users\JoyLal\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Data

In [2]:
iris = datasets.load_iris()

In [3]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(iris.data)

In [4]:
labels = [0 if target == 0 else 1 for target in  iris.target]

In [5]:
train_X, test_X, train_y, test_y = train_test_split(scaled_data, labels, test_size = 0.25, random_state=33)

# Neural Net

In [6]:
class NeuralNet(object):
    def __init__(self, hidden_nodes, learning_rate=0.01, output_nodes=1, epochs=1000):
        self.learning_rate = learning_rate
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        self.activation_function = self.sigmoid
        self.epochs = epochs
        
    def sigmoid(self, x):
        return 1.0 / (1.0 + np.exp(-x))
    
    def init_weights(self):
        self.weights_input_to_hidden = np.random.rand(self.n_attributes + 1, self.hidden_nodes)
        self.weights_hidden_to_output = np.random.rand(self.hidden_nodes + 1, self.output_nodes)
    
    def forward_pass(self, row):
        hidden_inputs = np.dot(row, self.weights_input_to_hidden)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        hidden_outputs = np.append(hidden_outputs, [1])
        
        final_inputs = np.dot(hidden_outputs, self.weights_hidden_to_output)
        final_outputs = self.activation_function(final_inputs)
        
        return hidden_outputs, final_outputs
    
    def backward_pass(self, row, hidden_outputs, final_outputs, target):
        final_error = final_outputs*(1 - final_outputs)*(target - final_outputs)
        hidden_error = hidden_outputs*(1 - hidden_outputs)*final_error*self.weights_hidden_to_output
        hidden_error = hidden_error[:-1]
        
        self.weights_hidden_to_output += self.learning_rate*np.dot(hidden_outputs, final_error)
        self.weights_input_to_hidden += self.learning_rate*np.dot(row.T, hidden_error.T)
        
    
    def train(self, inputs, targets):
        self.n_attributes = inputs.shape[1]
        self.init_weights()
        for i in range(0, self.epochs):
            for row, target in zip(inputs, targets):
                row = np.append(row, [1])
                row = row.reshape((1, self.n_attributes + 1))

                hidden_outputs, final_outputs = self.forward_pass(row)

                hidden_outputs = hidden_outputs.reshape((self.hidden_nodes + 1, 1))
                final_outputs = final_outputs.reshape((self.output_nodes, 1))

                self.backward_pass(row, hidden_outputs, final_outputs, target)
    
    def predict(self, inputs):
        results = []
        for row in inputs:
            row = np.append(row, [1])
            row = row.reshape((1, self.n_attributes + 1))
            hidden_outputs, final_outputs = self.forward_pass(row)
            results.append(round(final_outputs[0]))
        return results

In [7]:
NN = NeuralNet(hidden_nodes=3)

In [8]:
NN.train(train_X, train_y)

In [9]:
predictions = NN.predict(test_X)

In [10]:
accuracy_score(predictions, test_y)

1.0